In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import NuLattice.smearedInteraction as smearI
import NuLattice.CCM.coupled_cluster as ccm
import NuLattice.references as ref
import NuLattice.FCI.few_body_diagonalization as fbd
import NuLattice.lattice as lat
from scipy.sparse.linalg import eigsh as arpack_eigsh
import NuLattice.constants as consts
from opt_einsum import contract
import scipy.sparse as sparse

In [2]:
myL= 5
bpi = 0.7
sL = 0.08
sNL = 0.08
a = 1.0 / 100.0
at = 1.0 / 150.0
c0 = -1.85e-05/ (a ** 2)
a_lat = consts.hbarc * a
tKin, inter, trMat = smearI.get_full_int(myL, bpi, c0, sL, sNL, a, at, nk=3, method = 2)
holes = ref.ref_2H_gs
no2b_stuff = ccm.get_norm_ordered_ham(myL, holes, tKin, inter)

In [18]:
lattice = lat.get_lattice(myL)
inter = smearI.shortRangeV(lattice,myL, sL, sNL, c0)

Densities Generated
rho*f_SL Generated
Compressing 1
Compressing 2
Compressing 3
Compressing 4
Compressing 5
Converting to List
Summing Duplicates
Full Interaction Generated


In [19]:
print(len(inter))

1653488


In [17]:
# Compute the deuteron
# additive quantum numbers
numpart=2 # number of nucleons
tz = 0    # twice the value of isospin projection
sz = 2    # twice the value of spin projection
my_basis = lat.get_sp_basis(myL)
nstat =  len(my_basis)

# get two-body basis as a dictionary for lookup
H2_lookup = fbd.get_many_body_states(my_basis, numpart, total_tz=tz, total_sz=sz)
print("matrix dimension:", len(H2_lookup))

# make scipy.sparse.csr_matrix for kinetic energy 
T2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, tKin, nstat)
print("kinetic energy done")

# make scipy.sparse.csr_matrix for 2-body interactions 
V2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, inter, nstat)
print("2-body interaction done")

# add all into Hamiltonian
H2_csr_mat = T2_csr_mat + V2_csr_mat

# compute lowest eigenvalue(s)
k_eig=10 # number of eigenvalues
vals, vecs = arpack_eigsh(H2_csr_mat, k=k_eig, which='SA')
print("Energies (MeV):", vals / a)

matrix dimension: 15625
kinetic energy done
2-body interaction done
Energies (MeV): [1.26018678e-14 8.36304678e+00 8.36304678e+00 8.36304678e+00
 8.36304678e+00 8.36304678e+00 8.36304678e+00 8.36304678e+00
 8.36304678e+00 1.67260936e+01]


In [21]:
vals, vecs = arpack_eigsh(trMat, k=10, which='LA')
vals = -np.log(vals) / at / a
print(vals)

[50.18455854 50.18455854 45.18526462 25.01485084 25.01485084 25.01485084
 25.01370132 25.01370132 16.60366685 -5.13358175]
